In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [2]:
#chrome driver site : https://developer.chrome.com/docs/chromedriver/downloads?hl=pt-br

# Configuração do Selenium com ChromeDriver
def get_driver():
    chrome_options = Options()
    #chrome_options.add_argument("--headless")  # Executa sem abrir o navegador
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_service = ChromeService(executable_path='chromedriver.exe')
    return webdriver.Chrome(service=chrome_service, options=chrome_options)

In [4]:
# Função para carregar e processar a página
def scrape_with_selenium(keyword:str):
    link = f'https://store.usp.org/searchresults?Ntt=`{keyword.lower()}`*&Rdm=910&searchType=simple&type=search'
    driver = get_driver()
    try:
        # Acessar a página
        driver.get(link)

        # Esperar que um elemento carregue (ajuste o seletor de acordo com a página)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'attr-value'))
        )

        # Pegar o HTML carregado
        page_source = driver.page_source

        # Usar BeautifulSoup para tratar os dados
        soup = BeautifulSoup(page_source, 'html.parser')
        # get = soup.find_all('div',attrs={"class":"attr-value"})
        print(soup)
        # Aqui você pode processar os dados, por exemplo, encontrar elementos
        #dados = soup.find("span",attrs={"data-bind":"text:id"})
        #print(dados)

    finally:
        driver.quit()

In [12]:
# Função para carregar e processar a página
def scrape_with_Catalog(keyword:int):
    link = f'https://store.usp.org/product/{keyword}'
    driver = get_driver()
    try:
        # Acessar a página
        driver.get(link)

        # Esperar que um elemento carregue (ajuste o seletor de acordo com a página)
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'attr-value'))
        )

        # Pegar o HTML carregado
        page_source = driver.page_source

        # Usar BeautifulSoup para tratar os dados
        soup = BeautifulSoup(page_source, 'html.parser')
        get = soup.find_all('div',attrs={"class":"usp-certificates"})
        print(get)
        # Extrair os dados dos lotes e datas de validade
        lot_data = []
        for row in get.find('tbody').find_all('tr'):
            lot_number = row.find_all('td')[0].text.strip()
            valid_date = row.find_all('td')[2].text.strip()
            lot_data.append((lot_number, valid_date))

        # Exibir os resultados
        for lot, date in lot_data:
            print(f"Lote: {lot}, Data de Validade: {date}")

    finally:
        driver.quit()

In [13]:
keyword = '1085003'
scrape_with_Catalog(keyword)

[<div class="usp-certificates" id="usp-certificates">
<!--ko if: $parent.filteredLotItems()-->
<table class="table-responsive hidden-xs-screen">
<caption class="sr-only"></caption>
<thead>
<tr>
<th data-bind="widgetLocaleText: 'lotText'" scope="">Lot No</th>
<th data-bind="widgetLocaleText: 'certificateText'" scope="">USP Certificate or Product Information Sheet</th>
<th data-bind="widgetLocaleText: 'validDateText'" scope="">Valid Use Date</th>
<th data-bind="widgetLocaleText: 'originInfoText'" scope="">Country of Origin*</th>
<th data-bind="widgetLocaleText: 'materialOriginText'" scope="">Material Origin *</th>
</tr>
</thead>
<tbody>
<!--ko foreach: $parent.filteredLotItems() -->
<tr>
<td>
<!--ko if: lot_number -->
<span data-bind="text: lot_number">R160N0</span>
<!--ko if: ($data.current == 'true')-->
                                    (<span data-bind="widgetLocaleText: 'currentText'">Current</span>)
                                    <!--/ko-->
<!--/ko-->
<!--ko ifnot: lot_number